<a href="https://colab.research.google.com/github/AlessandraSalanitri/DataScienceProject/blob/main/CNN_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import zipfile
import os

# The folder downloaded from kaggle has been renamed CNN, compressed
# and uploaded to session storage.
# the path to the ZIP file below:
zip_path = "CNN.zip"
extract_path = "CNN_dataset"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted Files:")
print(os.listdir(extract_path))


Extracted Files:
['CNN']


In [20]:
import os

# path to the extracted folder
dataset_path = "CNN_dataset/CNN"

#list the contents of the dataset directory
print("Dataset Structure:", os.listdir(dataset_path))

#list the contents of each category folder
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    print(f"\nCategory: {category}")
    print(os.listdir(category_path)[:5])


Dataset Structure: ['covid', 'normal']

Category: covid
['6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg', '1-s2.0-S0929664620300449-gr2_lrg-d.jpg', 'F4341CE7-73C9-45C6-99C8-8567A5484B63.jpeg', 'auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg', 'ciaa199.pdf-001-b.png']

Category: normal
['IM-0131-0001.jpeg', 'NORMAL2-IM-1277-0001-0001.jpeg', 'IM-0387-0001.jpeg', 'NORMAL2-IM-0397-0001.jpeg', 'IM-0115-0001.jpeg']


In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# dataset path
dataset_path = "CNN_dataset/CNN"

# image properties
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Data Augmentation & Normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load Training Data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

# Load Validation Data
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

# Class Labels
print("\nClass Labels:", train_generator.class_indices)


Found 76 images belonging to 2 classes.
Found 18 images belonging to 2 classes.

Class Labels: {'covid': 0, 'normal': 1}


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN architecture
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(128, activation="relu"),
    Dropout(0.3),

    Dense(64, activation="relu"),
    Dropout(0.3),

    Dense(1, activation="sigmoid") # (COVID vs Normal)
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# model summary
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,312,961 (12.64 MB)

 Trainable params: 3,312,961 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# CNN model training
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# save trained model
model.save("covid_cnn_model.h5")

print("CNN Training Completed!")


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6593 - loss: 0.6460 - val_accuracy: 0.7222 - val_loss: 0.5815
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 886ms/step - accuracy: 0.7434 - loss: 0.5658 - val_accuracy: 0.7222 - val_loss: 0.5572
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.7171 - loss: 0.5539 - val_accuracy: 0.7222 - val_loss: 0.5483
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.7719 - loss: 0.5547 - val_accuracy: 0.7222 - val_loss: 0.4830
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.7932 - loss: 0.4503 - val_accuracy: 0.9444 - val_loss: 0.5179
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 882ms/step - accuracy: 0.8717 - loss: 0.4438 - val_accuracy: 0.7778 - val_loss: 0.3745
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 897ms/step - accuracy: 0.8520 - loss: 0.3967 - val_accuracy: 0.8889 - val_loss: 0.3625
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9200 - loss: 0.3455 - val_accuracy: 0.8333 - val_loss: 0.3664
Epoch

CNN Training Completed!
